# Pipeline 2: EEG-ARNN Methods - FINAL CORRECTED VERSION

**OPTIMIZED:** 20 epochs, 2-fold CV, Comprehensive metrics, Smart redundancy avoidance

## Models
1. Baseline-EEG-ARNN (without gating)
2. Adaptive-Gating-EEG-ARNN (YOUR CONTRIBUTION!)

## Workflow

### Part 1: Train Both Models on 64 Channels
- 2 models × 2 folds = 4 training runs (~1 hour)

### Part 2: Channel Selection Experiments
For each k in [10, 15, 20, 25, 30]:
  1. Extract importance scores using all 3 methods (Edge, Aggregation, Gate)
  2. Select top-k channels for each method
  3. **Compare channel sets** - if identical, train only once!
  4. Train model(s) on selected k channels
  5. Evaluate with comprehensive metrics

**Smart Optimization:** Skip redundant training when methods select same channels

## Metrics Calculated
- Accuracy, Precision, Recall (Sensitivity), F1-Score, AUC-ROC, Specificity

## Runtime: ~9-11 hours
- Initial training: ~1 hour (4 runs)
- Channel selection: ~8-10 hours (varies based on channel overlap)

## 1. Setup and Configuration

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import gc
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mne
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')
mne.set_log_level('ERROR')

print("All imports successful!")

In [ ]:
# Configuration
CONFIG = {
    'data_path': '/kaggle/input/eeg-preprocessed-data/derived',
    'models_dir': './models',
    'results_dir': './results',
    
    'n_folds': 2,
    'random_seed': 42,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # Training hyperparameters
    'batch_size': 32,
    'epochs': 20,
    'learning_rate': 0.002,
    'weight_decay': 1e-4,
    'patience': 5,
    'scheduler_patience': 2,
    'scheduler_factor': 0.5,
    'use_early_stopping': True,
    'min_lr': 1e-6,
    'use_amp': True,
    
    # Data parameters
    'n_channels': 64,
    'n_classes': 2,
    'sfreq': 128,
    'tmin': 0.0,
    'tmax': 4.0,
    'n_timepoints': 513,
    'hidden_dim': 128,
    'mi_runs': [7, 8, 11, 12],
    
    # Gating parameters
    'gating': {
        'gate_init': 0.9,
        'l1_lambda': 1e-3,
    },
    
    # Single k-values list for channel selection
    'k_values': [10, 15, 20, 25, 30],
}

os.makedirs(CONFIG['models_dir'], exist_ok=True)
os.makedirs(CONFIG['results_dir'], exist_ok=True)

np.random.seed(CONFIG['random_seed'])
torch.manual_seed(CONFIG['random_seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(CONFIG['random_seed'])

print(f"Device: {CONFIG['device']}")
print(f"Epochs: {CONFIG['epochs']}")
print(f"Folds: {CONFIG['n_folds']}")
print(f"Learning rate: {CONFIG['learning_rate']}")
print(f"k-values: {CONFIG['k_values']}")

# Runtime estimates
initial_runs = 2 * CONFIG['n_folds']
max_cs_runs = 2 * 3 * len(CONFIG['k_values']) * CONFIG['n_folds']

print(f"\nEstimated training runs:")
print(f"  Initial (64 channels): {initial_runs}")
print(f"  Channel selection (worst case): {max_cs_runs}")
print(f"  Channel selection (best case - with skips): ~{int(max_cs_runs * 0.7)}")
print(f"\nEstimated runtime: ~9-11 hours")

## 2. Metrics Calculation Functions

In [ ]:
def calculate_comprehensive_metrics(model, dataloader, device):
    """Calculate all metrics for PyTorch models."""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y_batch.numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    metrics = {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average='binary', zero_division=0),
        'recall': recall_score(all_labels, all_preds, average='binary', zero_division=0),
        'f1_score': f1_score(all_labels, all_preds, average='binary', zero_division=0),
        'auc_roc': roc_auc_score(all_labels, all_probs) if len(np.unique(all_labels)) > 1 else 0.0,
    }
    
    cm = confusion_matrix(all_labels, all_preds)
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        metrics['specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        metrics['sensitivity'] = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    else:
        metrics['specificity'] = 0.0
        metrics['sensitivity'] = 0.0
    
    return metrics


print("Metrics calculation function loaded!")

## 3. Data Loading

In [ ]:
def load_physionet_data(data_path):
    """Load preprocessed PhysioNet data."""
    data_root = os.path.abspath(data_path)
    if not os.path.isdir(data_root):
        raise FileNotFoundError(f"Data path not found: {data_root}")

    tmin, tmax = CONFIG['tmin'], CONFIG['tmax']
    mi_runs = CONFIG['mi_runs']
    event_id = {'T1': 1, 'T2': 2}
    label_map = {1: 0, 2: 1}

    preprocessed_dir = os.path.join(data_root, 'preprocessed')
    if os.path.isdir(preprocessed_dir):
        data_root = preprocessed_dir
    
    subject_dirs = [d for d in sorted(os.listdir(data_root))
                    if os.path.isdir(os.path.join(data_root, d)) and d.upper().startswith('S')]

    all_X, all_y, all_subjects = [], [], []
    print(f"Loading data from {len(subject_dirs)} subjects...")
    
    for subject_dir in subject_dirs:
        subject_num = int(subject_dir[1:]) if len(subject_dir) > 1 else -1
        subject_path = os.path.join(data_root, subject_dir)
        
        for run_id in mi_runs:
            run_file = f"{subject_dir}R{run_id:02d}_preproc_raw.fif"
            run_path = os.path.join(subject_path, run_file)
            
            if not os.path.exists(run_path):
                continue
            
            try:
                raw = mne.io.read_raw_fif(run_path, preload=True, verbose=False)
                picks = mne.pick_types(raw.info, eeg=True, meg=False, stim=False, eog=False)
                if len(picks) == 0:
                    continue
                
                events, _ = mne.events_from_annotations(raw, event_id=event_id)
                if len(events) == 0:
                    continue
                
                epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax,
                                    baseline=None, preload=True, picks=picks, verbose=False)
                
                data = epochs.get_data()
                labels = np.array([label_map.get(epochs.events[i, 2], -1) for i in range(len(epochs))])
                valid = labels >= 0
                
                if np.any(valid):
                    all_X.append(data[valid])
                    all_y.append(labels[valid])
                    all_subjects.append(np.full(np.sum(valid), subject_num))
            except:
                continue
    
    X = np.concatenate(all_X, axis=0)
    y = np.concatenate(all_y, axis=0)
    subjects = np.concatenate(all_subjects, axis=0)
    
    print(f"Loaded {len(X)} trials from {len(np.unique(subjects))} subjects")
    print(f"Data shape: {X.shape}")
    print(f"Labels: {np.bincount(y)}")
    
    return X, y, subjects


class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

## 4. Model Architectures

In [ ]:
# Graph Convolution Layer
class GraphConvLayer(nn.Module):
    def __init__(self, num_channels, hidden_dim):
        super().__init__()
        self.A = nn.Parameter(torch.randn(num_channels, num_channels) * 0.01)
        self.theta = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.bn = nn.BatchNorm2d(hidden_dim)
        self.act = nn.ELU()

    def forward(self, x):
        B, H, C, T = x.shape
        A = torch.sigmoid(self.A)
        A = 0.5 * (A + A.t())
        I = torch.eye(C, device=A.device)
        A_hat = A + I
        D = torch.diag(torch.pow(A_hat.sum(1).clamp_min(1e-6), -0.5))
        A_norm = D @ A_hat @ D

        x_perm = x.permute(0, 3, 2, 1).contiguous().view(B * T, C, H)
        x_g = A_norm @ x_perm
        x_g = self.theta(x_g)
        x_g = x_g.view(B, T, C, H).permute(0, 3, 2, 1)
        return self.act(self.bn(x_g))

    def get_adjacency(self):
        with torch.no_grad():
            A = torch.sigmoid(self.A)
            A = 0.5 * (A + A.t())
            return A.cpu().numpy()


# Temporal Convolution
class TemporalConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=16, pool=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=(1, kernel_size),
                              padding=(0, kernel_size // 2), bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ELU()
        self.pool_layer = nn.AvgPool2d(kernel_size=(1, 2)) if pool else None

    def forward(self, x):
        x = self.act(self.bn(self.conv(x)))
        if self.pool_layer is not None:
            x = self.pool_layer(x)
        return x

In [ ]:
# Baseline EEG-ARNN (without gating)
class BaselineEEGARNN(nn.Module):
    def __init__(self, n_channels=64, n_classes=2, n_timepoints=513, hidden_dim=128):
        super().__init__()
        self.n_channels = n_channels
        self.hidden_dim = hidden_dim
        self.use_gate_regularizer = False

        self.t1 = TemporalConv(1, hidden_dim, 16, pool=False)
        self.g1 = GraphConvLayer(n_channels, hidden_dim)
        self.t2 = TemporalConv(hidden_dim, hidden_dim, 16, pool=True)
        self.g2 = GraphConvLayer(n_channels, hidden_dim)
        self.t3 = TemporalConv(hidden_dim, hidden_dim, 16, pool=True)
        self.g3 = GraphConvLayer(n_channels, hidden_dim)

        with torch.no_grad():
            dummy = torch.zeros(1, n_channels, n_timepoints)
            feat = self._forward_features(self._prepare_input(dummy))
            self.feature_dim = feat.view(1, -1).size(1)

        self.fc1 = nn.Linear(self.feature_dim, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, n_classes)

    def _prepare_input(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(1)
        return x

    def _forward_features(self, x):
        x = self.g1(self.t1(x))
        x = self.g2(self.t2(x))
        x = self.g3(self.t3(x))
        return x

    def forward(self, x):
        prepared = self._prepare_input(x)
        features = self._forward_features(prepared)
        x = features.view(features.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

    def get_final_adjacency(self):
        return self.g3.get_adjacency()

    def get_channel_importance_edge(self):
        adjacency = self.get_final_adjacency()
        return np.sum(adjacency, axis=1)

In [ ]:
# Adaptive Gating EEG-ARNN (YOUR CONTRIBUTION!)
class AdaptiveGatingEEGARNN(BaselineEEGARNN):
    def __init__(self, n_channels=64, n_classes=2, n_timepoints=513, hidden_dim=128, gate_init=0.9):
        super().__init__(n_channels, n_classes, n_timepoints, hidden_dim)
        self.use_gate_regularizer = True
        
        self.gate_net = nn.Sequential(
            nn.Linear(n_channels * 2, n_channels),
            nn.ReLU(),
            nn.Linear(n_channels, n_channels),
            nn.Sigmoid()
        )
        
        init_value = float(np.clip(gate_init, 1e-3, 1 - 1e-3))
        init_bias = math.log(init_value / (1.0 - init_value))
        with torch.no_grad():
            self.gate_net[-2].bias.fill_(init_bias)
        
        self.latest_gate_values = None
        self.gate_penalty_tensor = None

    def compute_gates(self, x):
        x_s = x.squeeze(1)
        ch_mean = x_s.mean(dim=2)
        ch_std = x_s.std(dim=2)
        stats = torch.cat([ch_mean, ch_std], dim=1)
        return self.gate_net(stats)

    def forward(self, x):
        prepared = self._prepare_input(x)
        gates = self.compute_gates(prepared)
        self.gate_penalty_tensor = gates
        self.latest_gate_values = gates.detach()
        gated = prepared * gates.view(gates.size(0), 1, gates.size(1), 1)
        
        features = self._forward_features(gated)
        x = features.view(features.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

    def get_channel_importance_gate(self):
        if self.latest_gate_values is None:
            return None
        return self.latest_gate_values.mean(dim=0).cpu().numpy()

## 5. Training Utilities

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device, l1_lambda=0.0, scaler=None):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    
    use_amp = scaler is not None
    
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        # Mixed Precision Training
        if use_amp:
            with torch.cuda.amp.autocast():
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                
                gate_penalty = getattr(model, 'gate_penalty_tensor', None)
                if l1_lambda > 0 and gate_penalty is not None:
                    loss = loss + l1_lambda * gate_penalty.abs().mean()
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            gate_penalty = getattr(model, 'gate_penalty_tensor', None)
            if l1_lambda > 0 and gate_penalty is not None:
                loss = loss + l1_lambda * gate_penalty.abs().mean()
            
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
        
        # Aggressive memory cleanup
        del outputs, loss
    
    return total_loss / max(1, len(dataloader)), correct / max(1, total)


def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
        
        # Aggressive memory cleanup
        del outputs, loss
    
    return total_loss / max(1, len(dataloader)), correct / max(1, total)


def train_pytorch_model(model, train_loader, val_loader, config, model_name=''):
    device = config['device']
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'], 
                          weight_decay=config['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=config['scheduler_factor'], 
        patience=config['scheduler_patience'], min_lr=config['min_lr']
    )
    
    l1_lambda = config['gating']['l1_lambda'] if getattr(model, 'use_gate_regularizer', False) else 0.0
    
    # Initialize Scaler for AMP
    scaler = torch.cuda.amp.GradScaler() if config.get('use_amp', False) and device != 'cpu' else None
    
    best_state = {k: v.cpu() if hasattr(v, 'cpu') else v for k, v in model.state_dict().items()}
    best_val_acc = 0.0
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(config['epochs']):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, l1_lambda, scaler)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        scheduler.step(val_loss)
        
        improved = val_acc > best_val_acc or (val_acc == best_val_acc and val_loss < best_val_loss)
        if improved:
    best_state = {k: v.cpu() if hasattr(v, 'cpu') else v for k, v in model.state_dict().items()}
            best_val_acc = val_acc
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if epoch % 5 == 0 or improved:
            print(f"[{model_name}] Epoch {epoch+1}/{config['epochs']} - "
                  f"Train: {train_acc:.4f} | Val: {val_acc:.4f} | Best: {best_val_acc:.4f}")
        
        if config['use_early_stopping'] and patience_counter >= config['patience']:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    model.load_state_dict(best_state)
    return best_state, model


## 6. Channel Selection Utilities

In [ ]:
def get_channel_importance_aggregation(model, dataloader, device):
    """Aggregation Selection: based on feature activations."""
    model.eval()
    channel_stats = []

    with torch.no_grad():
        for X_batch, _ in dataloader:
            X_batch = X_batch.to(device)
            prepared = model._prepare_input(X_batch)
            features = model._forward_features(prepared)
            activations = torch.mean(torch.abs(features), dim=(1, 3))
            channel_stats.append(activations.cpu())

    if not channel_stats:
        return np.zeros(model.n_channels)
    stacked = torch.cat(channel_stats, dim=0)
    return stacked.mean(dim=0).numpy()


def compute_gate_importance(model, dataloader, device):
    """Gate Selection: average gate values across dataset."""
    model.eval()
    gate_batches = []

    with torch.no_grad():
        for X_batch, _ in dataloader:
            X_batch = X_batch.to(device)
            _ = model(X_batch)
            latest = getattr(model, 'latest_gate_values', None)
            if latest is not None:
                gate_batches.append(latest.cpu())

    if not gate_batches:
        return np.ones(model.n_channels) / model.n_channels
    stacked = torch.cat(gate_batches, dim=0)
    return stacked.mean(dim=0).numpy()


def select_top_k_channels(importance_scores, k):
    """Select top k channels and return as sorted list."""
    top_k_indices = np.argsort(importance_scores)[-k:]
    return sorted(top_k_indices.tolist())


def apply_channel_selection(X, selected_channels):
    """Apply channel selection to data."""
    return X[:, selected_channels, :]

## 7. Load Data

In [ ]:
print("Loading PhysioNet data...")
X, y, subjects = load_physionet_data(CONFIG['data_path'])
print(f"\nData ready!")

## 8. Train Initial Models (64 Channels)

In [ ]:
# Setup cross-validation
skf = StratifiedKFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['random_seed'])

models_to_train = [
    {'name': 'Baseline-EEG-ARNN', 'class': BaselineEEGARNN},
    {'name': 'Adaptive-Gating-EEG-ARNN', 'class': AdaptiveGatingEEGARNN},
]

print(f"\n{'='*60}")
print("TRAINING INITIAL MODELS (64 CHANNELS)")
print(f"{'='*60}\n")

all_results = {}

In [ ]:
# Training loop for 64-channel models
for model_info in models_to_train:
    model_name = model_info['name']
    model_class = model_info['class']
    
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}\n")
    
    fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\nFold {fold + 1}/{CONFIG['n_folds']}")
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        train_dataset = EEGDataset(X_train, y_train)
        val_dataset = EEGDataset(X_val, y_val)
        train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False)
        
        if model_class == AdaptiveGatingEEGARNN:
            model = model_class(
                n_channels=CONFIG['n_channels'],
                n_classes=CONFIG['n_classes'],
                n_timepoints=CONFIG['n_timepoints'],
                hidden_dim=CONFIG['hidden_dim'],
                gate_init=CONFIG['gating']['gate_init']
            )
        else:
            model = model_class(
                n_channels=CONFIG['n_channels'],
                n_classes=CONFIG['n_classes'],
                n_timepoints=CONFIG['n_timepoints'],
                hidden_dim=CONFIG['hidden_dim']
            )
        
        best_state, model = train_pytorch_model(model, train_loader, val_loader, CONFIG, model_name)
        
        # Calculate comprehensive metrics
        metrics = calculate_comprehensive_metrics(model, val_loader, CONFIG['device'])
        
        # Save model
        model_path = os.path.join(CONFIG['models_dir'], f"eegarnn_{model_name}_fold{fold+1}.pt")
        torch.save(best_state, model_path)
        
        result = {'fold': fold + 1}
        result.update(metrics)
        fold_results.append(result)
        
        print(f"\nFold {fold + 1} Results:")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  F1-Score: {metrics['f1_score']:.4f}")
        print(f"  AUC-ROC: {metrics['auc_roc']:.4f}")
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
    
    all_results[model_name] = fold_results
    
    mean_acc = np.mean([r['accuracy'] for r in fold_results])
    print(f"\n{model_name} Mean Accuracy: {mean_acc:.4f}")

print(f"\n{'='*60}")
print("INITIAL MODELS TRAINED!")
print(f"{'='*60}")

## 9. Channel Selection Experiments

In [ ]:
print(f"\n{'='*60}")
print("CHANNEL SELECTION EXPERIMENTS")
print(f"{'='*60}\n")

channel_selection_results = []
training_count = 0
skipped_count = 0

In [ ]:
# Channel selection experiments
for model_info in models_to_train:
    model_name = model_info['name']
    model_class = model_info['class']
    
    # Define which methods apply to this model
    if 'Baseline' in model_name:
        methods = ['edge', 'aggregation']
    else:
        methods = ['edge', 'aggregation', 'gate']
    
    print(f"\n{'='*60}")
    print(f"{model_name}")
    print(f"{'='*60}\n")
    
    for k in CONFIG['k_values']:
        print(f"\nk={k} channels:")
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            # Load the trained 64-channel model
            if model_class == AdaptiveGatingEEGARNN:
                full_model = model_class(
                    n_channels=CONFIG['n_channels'],
                    n_classes=CONFIG['n_classes'],
                    n_timepoints=CONFIG['n_timepoints'],
                    hidden_dim=CONFIG['hidden_dim'],
                    gate_init=CONFIG['gating']['gate_init']
                )
            else:
                full_model = model_class(
                    n_channels=CONFIG['n_channels'],
                    n_classes=CONFIG['n_classes'],
                    n_timepoints=CONFIG['n_timepoints'],
                    hidden_dim=CONFIG['hidden_dim']
                )
            
            model_path = os.path.join(CONFIG['models_dir'], f"eegarnn_{model_name}_fold{fold+1}.pt")
            state_dict = torch.load(model_path, map_location=CONFIG['device'])
            full_model.load_state_dict(state_dict)
            full_model = full_model.to(CONFIG['device'])
            full_model.eval()
            
            # Get channel importance for all methods
            train_dataset = EEGDataset(X_train, y_train)
            train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=False)
            
            method_channels = {}
            for method in methods:
                if method == 'edge':
                    importance = full_model.get_channel_importance_edge()
                elif method == 'aggregation':
                    importance = get_channel_importance_aggregation(full_model, train_loader, CONFIG['device'])
                else:  # gate
                    importance = compute_gate_importance(full_model, train_loader, CONFIG['device'])
                
                selected = select_top_k_channels(importance, k)
                method_channels[method] = selected
            
            del full_model
            torch.cuda.empty_cache()
            gc.collect()
            
            # Group methods by identical channel sets to avoid redundant training
            unique_channel_sets = {}
            for method in methods:
                channels_tuple = tuple(method_channels[method])
                if channels_tuple not in unique_channel_sets:
                    unique_channel_sets[channels_tuple] = []
                unique_channel_sets[channels_tuple].append(method)
            
            # Train only once per unique channel set
            for channels_tuple, method_group in unique_channel_sets.items():
                channels_list = list(channels_tuple)
                
                if len(method_group) > 1:
                    print(f"  Fold {fold+1}: Methods {method_group} selected same channels - training once")
                    skipped_count += len(method_group) - 1
                
                # Apply channel selection
                X_train_selected = apply_channel_selection(X_train, channels_list)
                X_val_selected = apply_channel_selection(X_val, channels_list)
                
                # Train model with k channels
                train_dataset = EEGDataset(X_train_selected, y_train)
                val_dataset = EEGDataset(X_val_selected, y_val)
                train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False)
                
                if model_class == AdaptiveGatingEEGARNN:
                    new_model = model_class(
                        n_channels=k,
                        n_classes=CONFIG['n_classes'],
                        n_timepoints=CONFIG['n_timepoints'],
                        hidden_dim=CONFIG['hidden_dim'],
                        gate_init=CONFIG['gating']['gate_init']
                    )
                else:
                    new_model = model_class(
                        n_channels=k,
                        n_classes=CONFIG['n_classes'],
                        n_timepoints=CONFIG['n_timepoints'],
                        hidden_dim=CONFIG['hidden_dim']
                    )
                
                best_state, new_model = train_pytorch_model(
                    new_model, train_loader, val_loader, CONFIG, 
                    f"{model_name}-{'+'.join(method_group)}-k{k}-fold{fold+1}"
                )
                
                training_count += 1
                
                # Calculate comprehensive metrics
                metrics = calculate_comprehensive_metrics(new_model, val_loader, CONFIG['device'])
                
                # Save results for all methods in the group
                for method in method_group:
                    result = {
                        'model': model_name,
                        'method': method,
                        'k': k,
                        'fold': fold + 1,
                        'selected_channels': str(channels_list)
                    }
                    result.update(metrics)
                    channel_selection_results.append(result)
                
                del new_model
                torch.cuda.empty_cache()
                gc.collect()
        
        # Print summary for this k
        k_results = [r for r in channel_selection_results 
                     if r['model'] == model_name and r['k'] == k]
        for method in methods:
            method_results = [r for r in k_results if r['method'] == method]
            if method_results:
                mean_acc = np.mean([r['accuracy'] for r in method_results])
                print(f"  {method}: {mean_acc:.4f}")

print(f"\n{'='*60}")
print("CHANNEL SELECTION COMPLETE!")
print(f"{'='*60}")
print(f"\nTotal trainings performed: {training_count}")
print(f"Trainings skipped (redundant): {skipped_count}")
print(f"Time saved: ~{skipped_count * 10 / 60:.1f} hours")

## 10. Save All Results

In [ ]:
# Save initial 64-channel results
for model_name, fold_results in all_results.items():
    detailed_results = []
    for result in fold_results:
        detailed_results.append({
            'model': model_name,
            'fold': result['fold'],
            'accuracy': result['accuracy'],
            'precision': result['precision'],
            'recall': result['recall'],
            'f1_score': result['f1_score'],
            'auc_roc': result['auc_roc'],
            'specificity': result['specificity'],
            'sensitivity': result['sensitivity']
        })
    
    df = pd.DataFrame(detailed_results)
    filename = model_name.lower().replace('-', '_').replace(' ', '_')
    df.to_csv(os.path.join(CONFIG['results_dir'], f'eegarnn_{filename}_results.csv'), index=False)
    print(f"Saved: results/eegarnn_{filename}_results.csv")

In [ ]:
# Save channel selection results
cs_df = pd.DataFrame(channel_selection_results)
cs_df.to_csv(os.path.join(CONFIG['results_dir'], 'channel_selection_results.csv'), index=False)

print("\nChannel Selection Results (sample):")
print(cs_df[['model', 'method', 'k', 'fold', 'accuracy', 'f1_score', 'auc_roc']].head(20))
print("\nSaved: results/channel_selection_results.csv")

## 11. Final Summary

In [ ]:
print(f"\n{'='*60}")
print("COMPLETE SUMMARY")
print(f"{'='*60}\n")

# Initial models (64 channels)
print("1. Initial Models (64 channels):")
for model_name, fold_results in all_results.items():
    accs = [r['accuracy'] for r in fold_results]
    f1s = [r['f1_score'] for r in fold_results]
    print(f"   {model_name}:")
    print(f"     Accuracy: {np.mean(accs):.4f} +/- {np.std(accs):.4f}")
    print(f"     F1-Score: {np.mean(f1s):.4f} +/- {np.std(f1s):.4f}")

# Best channel selection results
print("\n2. Best Channel Selection Results:")
for k in CONFIG['k_values']:
    k_results = cs_df[cs_df['k'] == k]
    best_idx = k_results['accuracy'].idxmax()
    best = k_results.loc[best_idx]
    print(f"   k={k}: {best['method']} ({best['model']}) - Acc={best['accuracy']:.4f}")

# Method comparison
print("\n3. Method Comparison (averaged across all k):")
for method in ['edge', 'aggregation', 'gate']:
    method_results = cs_df[cs_df['method'] == method]
    if len(method_results) > 0:
        mean_acc = method_results['accuracy'].mean()
        print(f"   {method.capitalize()}: {mean_acc:.4f}")

print(f"\n{'='*60}")
print("PIPELINE 2 COMPLETE!")
print(f"{'='*60}")
print("\nAll comprehensive metrics calculated and saved.")
print("\nOutputs:")
print("  1. eegarnn_baseline_eeg_arnn_results.csv")
print("  2. eegarnn_adaptive_gating_eeg_arnn_results.csv")
print("  3. channel_selection_results.csv")
print("\nReady for comparison with Pipeline 1!")